# Integrantes



*   Adrián Landaverde Nava
*   Cristian Gonzaga López
*   Sabrina Nicole Rodriguez Salgado
*   Naomi Padilla Mora



# 01. Librerías y Datos iniciales

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random
from shapely.geometry import Point
import math
import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium import Circle
import numpy as np

In [ ]:
mapa = gpd.read_file("/content/drive/MyDrive/Universidad/Tercer Semestre/Modelación con Ecuaciones Diferenciales/México_Estados.shp")
mapa= mapa.sort_values('ESTADO')
mapa.head(32)

In [ ]:
poblacion= pd.read_csv("/content/drive/MyDrive/Universidad/Tercer Semestre/Modelación con Ecuaciones Diferenciales/poblacionEstados.csv",encoding="latin-1")
poblacion.head()

In [ ]:
mapa.index= list(range(0,32))
mapa["POBLACION"]=poblacion["Ppoblacion"]
mapa.head()

In [ ]:
mapa.crs = {'init': 'epsg:4326'}

In [ ]:
from shapely.ops import unary_union
mapaMexico = unary_union(mapa["geometry"])
print(mapaMexico)
mapaMexico

# 02. Creación de puntos

## A) Población distribuida uniformemente

In [ ]:
def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
            if(len(points)%10000==0):
              print(str(len(points))+" puntos simulados")
    return points

In [ ]:
df_aleatorios= pd.DataFrame(columns=["geometry"])
df_aleatorios["geometry"]=generate_random(10000,mapaMexico)
df_aleatorios.head()

In [ ]:
gdf_aleatorios= gpd.GeoDataFrame(df_aleatorios)
gdf_aleatorios['lon'] = gdf_aleatorios.geometry.x
gdf_aleatorios['lat'] = gdf_aleatorios.geometry.y
gdf_aleatorios.head()

In [ ]:
figure(figsize=(15, 10))
plt.scatter(gdf_aleatorios['lon'],gdf_aleatorios['lat'],alpha=0.5,s=3)

In [ ]:
gdf_aleatoriosA=gdf_aleatorios

## B) Población distribuida uniformemente por Estado

In [ ]:
def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

In [ ]:
df_aleatorios= pd.DataFrame(columns=["geometry"])
df_aleatorios
for i in range(0,32):
  aleatorios= generate_random(round(mapa["POBLACION"][i]/10000),mapa.iloc[i]['geometry'])
  df = pd.DataFrame(aleatorios,columns=["geometry"])
  df_aleatorios= df_aleatorios.append(df)
  print("Estados terminados: "+str(i+1)+ "   Población Acumulada= " +str(df_aleatorios.size))
print("Población Total=" +str(df_aleatorios.size))


In [ ]:
gdf_aleatorios= gpd.GeoDataFrame(df_aleatorios)
gdf_aleatorios['lon'] = gdf_aleatorios.geometry.x
gdf_aleatorios['lat'] = gdf_aleatorios.geometry.y
gdf_aleatorios.head()

In [ ]:
figure(figsize=(15, 10))
plt.scatter(gdf_aleatorios['lon'],gdf_aleatorios['lat'],alpha=0.5,s=3)

In [ ]:
gdf_aleatoriosB=gdf_aleatorios

## C) Ciudades Distribuidas uniformemente

In [ ]:
def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

In [ ]:
n_ciudades= 500
df_aleatorios= pd.DataFrame(columns=["geometry"])
df_aleatorios["geometry"]=generate_random(n_ciudades,mapaMexico)
df_aleatorios.head()

In [ ]:
gdf_aleatorios= gpd.GeoDataFrame(df_aleatorios)
gdf_aleatorios['lon'] = gdf_aleatorios.geometry.x
gdf_aleatorios['lat'] = gdf_aleatorios.geometry.y
gdf_aleatorios.crs = {'init': 'epsg:4326'}
gdf_aleatorios.head()

In [ ]:
radioCiudad= 50 #Kilómetros
bufferCiudades = gdf_aleatorios.to_crs(epsg=2272).geometry.buffer(radioCiudad*3280.84)
figure(figsize=(15, 10))
bufferCiudades.to_crs(epsg=4326).plot()

In [ ]:
df_aleatorios2= pd.DataFrame(columns=["geometry"])
for i in range(0,n_ciudades):
  aleatorios= generate_random(10000/n_ciudades,bufferCiudades[i])
  df = pd.DataFrame(aleatorios,columns=["geometry"])
  df_aleatorios2= df_aleatorios2.append(df)
  if((i+1)%(n_ciudades/10)==0):
    print(str((i+1)/(n_ciudades))+"% completo")
print("Población Total=" +str(df_aleatorios2.size))

In [ ]:
gdf_aleatorios2= gpd.GeoDataFrame(df_aleatorios2)
gdf_aleatorios2.crs = {'init': 'epsg:2272'}
gdf_aleatorios2= gdf_aleatorios2.to_crs(epsg=4326)
gdf_aleatorios2['lon'] = gdf_aleatorios2.geometry.x
gdf_aleatorios2['lat'] = gdf_aleatorios2.geometry.y
gdf_aleatorios2.head()

In [ ]:
figure(figsize=(15, 10))
plt.scatter(gdf_aleatorios2['lon'],gdf_aleatorios2['lat'],alpha=0.5,s=3)

In [ ]:
gdf_aleatoriosC=gdf_aleatorios2

## D) Ciudades distribuidas uniformemente por Estado

In [ ]:
def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

In [ ]:
poblacionPorCiudad= 20
df_aleatorios= pd.DataFrame(columns=["geometry"])
df_aleatorios
for i in range(0,32):
  aleatorios= generate_random(round(mapa["POBLACION"][i]/(10000*poblacionPorCiudad)),mapa.iloc[i]['geometry'])
  df = pd.DataFrame(aleatorios,columns=["geometry"])
  df_aleatorios= df_aleatorios.append(df)
  print("Estados terminados: "+str(i+1)+ "   Ciudades Acumuladas= " +str(df_aleatorios.size))
print("Ciudades Totales=" +str(df_aleatorios.size))

In [ ]:
gdf_aleatorios= gpd.GeoDataFrame(df_aleatorios)
gdf_aleatorios['lon'] = gdf_aleatorios.geometry.x
gdf_aleatorios['lat'] = gdf_aleatorios.geometry.y
gdf_aleatorios.crs = {'init': 'epsg:4326'}
gdf_aleatorios.head()

In [ ]:
gdf_aleatorios.index=list(range(0,len(gdf_aleatorios)))

In [ ]:
radioCiudad= 50 #Kilómetros
bufferCiudades = gdf_aleatorios.to_crs(epsg=2272).geometry.buffer(radioCiudad*3280.84)
figure(figsize=(15, 10))
bufferCiudades.to_crs(epsg=4326).plot()

In [ ]:
df_aleatorios2= pd.DataFrame(columns=["geometry"])
for i in range(0,len(bufferCiudades)):
  aleatorios= generate_random(poblacionPorCiudad,bufferCiudades[i])
  df = pd.DataFrame(aleatorios,columns=["geometry"])
  df_aleatorios2= df_aleatorios2.append(df)
print("Población Total=" +str(df_aleatorios2.size))

In [ ]:
gdf_aleatorios2= gpd.GeoDataFrame(df_aleatorios2)
gdf_aleatorios2.crs = {'init': 'epsg:2272'}
gdf_aleatorios2= gdf_aleatorios2.to_crs(epsg=4326)
gdf_aleatorios2['lon'] = gdf_aleatorios2.geometry.x
gdf_aleatorios2['lat'] = gdf_aleatorios2.geometry.y
gdf_aleatorios2.head()

In [ ]:
figure(figsize=(15, 10))
plt.scatter(gdf_aleatorios2['lon'],gdf_aleatorios2['lat'],alpha=0.5,s=3)

In [ ]:
gdf_aleatoriosD=gdf_aleatorios2

# 03. Condiciones iniciales de la Infección

In [ ]:
gdf_aleatorioF= gdf_aleatoriosA
#gdf_aleatorioF= gdf_aleatoriosB
#gdf_aleatorioF= gdf_aleatoriosC
#gdf_aleatorioF= gdf_aleatoriosD
gdf_aleatorioF.head()

In [ ]:
#Condiciones iniciales:
N= len(gdf_aleatorioF)
I0  = 1 #número infectados inicial
R0 = 0 #número recuperados inicial
D0 = 0 #número muertos inicial
S0 = N-I0-D0-R0 #Número de susceptibles inicial

infection_radius = 0.1  #distancia para decidir si una persona se infecta o no
recovery_rate = 0.01
death_rate = 0.0001

### Definir un array donde aparezca un número de elementos "susceptible" igual a S0, un número de elementos "infected" igual a I0,
### un número de elementos "recovered" igual a R0 y un número de elementos "dead" igual a D0
## Sug.:usar np.repeat()
sus= np.repeat("suceptible",S0)
inf= np.repeat("infected",I0)
rec= np.repeat("recovered",R0)
dead= np.repeat("dead",D0)
status = np.concatenate((sus,inf,rec,dead))
ststaus= random.shuffle(status)

### Crear un array de N elementos iguales a 0 (usar np.repeat())
#iteration = np.repeat(0,N)
### Crear un array de N elementos que contenga valores de 1 a N
id = list(range(0,N))

In [ ]:
gdf_aleatorioF["status"]=status
#gdf_aleatorioF["iteration"]=iteration
gdf_aleatorioF["id"]=id
gdf_aleatorioF.index=gdf_aleatorioF["id"]
gdf_aleatorioF=gdf_aleatorioF.drop(["id"],axis=1)
gdf_aleatorioF.rename(columns={'geometry': 'geometry0', 'lon': 'lon0', 'lat':'lat0','status':'status0'}, inplace=True)
gdf_aleatorioF

In [ ]:
colors = {'recovered':'green', 'infected':'red', 'dead':'black', 'suceptible':'blue'}
sDia = dict(size=30, color='black')
sSuceptibles = dict(size=25, color='blue')
sInfectados = dict(size=25, color='red')
sRecuperados = dict(size=25, color='green')
sFallecidos = dict(size=25, color='black')
fig, ax = plt.subplots(figsize=(15, 10))
ax.scatter(gdf_aleatorioF['lon0'],gdf_aleatorioF['lat0'],c=gdf_aleatorioF["status0"].map(colors),alpha=0.5,s=3)
ax.text(-103, 32, ("Día: 0"),**sDia)
ax.text(-118, 21, ("Suceptibles: "+str(S0)),**sSuceptibles)
ax.text(-118, 19, ("Infectados: "+str(I0)),**sInfectados)
ax.text(-118, 17, ("Recuperados: "+str(R0)),**sRecuperados)
ax.text(-118, 15, ("Fallecidos: "+str(D0)),**sFallecidos)
ax.set_xlim(-120, -85)
ax.set_ylim(14, 34)
plt.show()

In [ ]:
def infec(lonA,latA,lonB,latB):
  distancia= np.sqrt(np.power(list(np.array(lonA)-np.array(lonB)),2)+np.power(list(np.array(latA)-np.array(latB)),2))
  return ((distancia<infection_radius))

# 04. Movimiento Aleatorio

In [ ]:
pd.set_option('mode.chained_assignment', None)

In [ ]:
gdf_aleatorioF['statusN']=gdf_aleatorioF['status0']
for i in range(1,51):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()





In [ ]:
for i in range(51,101):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()


In [ ]:
for i in range(101,151):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()

In [ ]:
for i in range(151,201):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()

In [ ]:
for i in range(201,251):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()

In [ ]:
for i in range(251,301):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()

In [ ]:
for i in range(301,366):
  # Movimiento Aleatorio
  randLon= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  randLat= (np.random.random_sample(size = len(gdf_aleatorioF))-0.5)*0.5
  gdf_aleatorioF['lonN']=gdf_aleatorioF['lon0']+randLon
  gdf_aleatorioF['latN']=gdf_aleatorioF['lat0']+randLat
  

  # Recuperación
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randRec= (np.random.random_sample(size = len(df_infectados)))
  boolsRec=(randRec<recovery_rate)
  indicesRec= df_infectados.loc[boolsRec]
  indicesRec= indicesRec.index.to_list()
  gdf_aleatorioF.loc[indicesRec,["statusN"]]="recovered"

  # Fallecimiento
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  randMuer= (np.random.random_sample(size = len(df_infectados)))
  boolsMuer=(randMuer<death_rate)
  indicesMuer= df_infectados.loc[boolsMuer]
  indicesMuer= indicesMuer.index.to_list()
  gdf_aleatorioF.loc[indicesMuer,["statusN"]]="dead"

  
  # Transmisión de la enfermedad
  df_infectados= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "infected"]
  df_suceptibles= gdf_aleatorioF[gdf_aleatorioF['statusN'] == "suceptible"]
  for j in df_infectados.index.to_list():
    iteracionInf=df_infectados.iloc[df_infectados.index.get_loc(j)]
    bools= infec(iteracionInf['lonN'],iteracionInf['latN'],list(df_suceptibles['lonN']),list(df_suceptibles['latN']))
    df_suceptibles["contagiado"]=bools
    indices= df_suceptibles.loc[bools]
    indices= indices.index.to_list()
    gdf_aleatorioF.loc[indices,["statusN"]]="infected"
  nS= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "suceptible"])
  nI= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "infected"])
  nR= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "recovered"])
  nD= len(gdf_aleatorioF[gdf_aleatorioF["statusN"] == "dead"])
  fig, ax = plt.subplots(figsize=(15, 10))
  ax.scatter(gdf_aleatorioF['lonN'],gdf_aleatorioF['latN'],c=gdf_aleatorioF["statusN"].map(colors),alpha=0.5,s=3)
  ax.text(-103, 32, ("Día: "+str(i)),**sDia)
  ax.text(-118, 21, ("Suceptibles: "+str(nS)),**sSuceptibles)
  ax.text(-118, 19, ("Infectados: "+str(nI)),**sInfectados)
  ax.text(-118, 17, ("Recuperados: "+str(nR)),**sRecuperados)
  ax.text(-118, 15, ("Fallecidos: "+str(nD)),**sFallecidos)
  ax.set_xlim(-120, -85)
  ax.set_ylim(14, 34)
  plt.show()